In [9]:
import time
import csv
import re
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
global browser 
browser = webdriver.Chrome(executable_path=r"J:\Projects\chrome\chromedriver.exe")
browser.get("https://www.whoscored.com/")
soup = BeautifulSoup(browser.page_source, 'html.parser')

In [2]:
def getTeamInfo(team):
    teamInfo=[]
    timeout=100
    browser.get("https://www.whoscored.com/"+team.a['href'])
    element_present = EC.presence_of_element_located((By.CLASS_NAME, 'rating'))
    WebDriverWait(browser, timeout).until(element_present)
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    time.sleep(2)
    #GET TEAM TITLE
    teamName=soup.find('span', class_ = 'team-header-name').getText()
    
    #GET TEAM RATING
    teamInfo.append(soup.find('div', class_ = 'rating').getText())
    
    #GET LAST 6 GAMES RESULT
    game=soup.find_all('tr', class_ = 'item')
    for i in range(0,6):
        try:
            score=list(map(int, game[i].find("a",class_ ="result-1 rc").getText().replace('*','').split(":")))
            if(game[i].find("a",class_ ="team-link").getText()==teamName):
                teamInfo.append(score[0])
                teamInfo.append(score[1])
            else:
                teamInfo.append(score[1])
                teamInfo.append(score[0])
        except:
            teamInfo.append(0)
            teamInfo.append(0)
            
    #GET TEAM AVERAGE STATS IN LAST GAMES
    teamAverage=soup.find('dl', class_ = 'stats').find_all('dd')
    teamInfo.append(teamAverage[2].getText())
    teamInfo.append(teamAverage[3].getText().replace('%',''))
    teamInfo.append(teamAverage[4].getText().replace('%',''))
    teamInfo.append(teamAverage[5].getText())
    #GET TEAM PLAYERS RATINGS
    playerList=soup.find_all('span', {'class':['player-rating rc', 'player-rating rc formOverlay-marker']})
    for player in playerList:
        teamInfo.append(re.sub('[^0-9.]','', player.getText()[:4]))
    time.sleep(1)
    print(teamInfo)
    return teamInfo

In [3]:
games_list = soup.find_all('tr', class_ = 'match')
gameTFile=[]
gamesToReport=[]

In [6]:
for game in games_list[:-2]:
    gamesToReport.append([game.find('td', class_ = 'team home').a.getText(),"VS",game.find('td', class_ = 'team away').a.getText()])
    gameTFile.append(getTeamInfo(game.find('td', class_ = 'team home')) + getTeamInfo(game.find('td', class_ = 'team away')))

['6.68', 0, 0, 1, 0, 2, 1, 0, 1, 1, 1, 0, 3, '1.2', '48.5', '77.4', '12.6', '6.5', '6.7', '7.3', '6.5', '7.1', '6.9', '6.7', '6.4', '7.5', '7.3', '7']
['6.63', 1, 2, 1, 1, 0, 1, 0, 2, 0, 3, 2, 0, '1.1', '46.1', '77.1', '12', '6.5', '7.1', '7', '6.9', '6.7', '6.6', '7.1', '6.9', '6.8', '6.8', '7.5']
['6.89', 0, 0, 0, 1, 1, 2, 3, 0, 1, 1, 1, 4, '1.9', '55', '82.7', '13.9', '6.7', '7.5', '7', '7.3', '6.8', '7', '6.8', '7', '7.1', '7.3', '7.1']
['6.75', 0, 3, 1, 2, 3, 1, 1, 1, 1, 0, 3, 0, '1.1', '44.5', '74.4', '9.5', '6.7', '6.8', '6.8', '6.9', '6.9', '6.9', '6.8', '6.9', '7', '7.2', '6.8']
['6.69', 2, 0, 0, 4, 0, 1, 0, 3, 2, 3, 0, 2, '0.9', '46.4', '76.3', '9.8', '6.4', '6.5', '6.5', '6.7', '6.7', '6.6', '6.6', '6.8', '6.9', '6.7', '7']
['6.82', 3, 0, 2, 1, 3, 0, 5, 2, 3, 0, 2, 2, '1.7', '54.5', '83.4', '14.5', '6.4', '7.1', '7', '7.1', '6.7', '7', '6.7', '7.2', '7.4', '8.1', '7.4']
['6.69', 3, 0, 0, 2, 3, 1, 1, 0, 1, 1, 2, 2, '1.3', '48.1', '72.5', '12.8', '6.7', '6.8', '6.7', '7.3', '6

In [19]:
with open('games.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for game in gameTFile:
        writer.writerow(game)
with open('results.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["t1","VS","t2"])
    for game in gamesToReport:
        writer.writerow(game)

In [18]:
print(gamesToReport)

[['Everton', 'VS', 'Aston Villa'], ['Leicester', 'VS', 'Sheffield United'], ['Crystal Palace', 'VS', 'Manchester United'], ['Southampton', 'VS', 'Brighton'], ['Eibar', 'VS', 'Real Valladolid'], ['Getafe', 'VS', 'Atletico Madrid'], ['Mallorca', 'VS', 'Granada'], ['Real Madrid', 'VS', 'Villarreal'], ['Real Sociedad', 'VS', 'Sevilla'], ['Valencia', 'VS', 'Espanyol'], ['Athletic Bilbao', 'VS', 'Leganes'], ['Barcelona', 'VS', 'Osasuna'], ['Real Betis', 'VS', 'Deportivo Alaves'], ['Celta Vigo', 'VS', 'Levante'], ['Torino', 'VS', 'Genoa'], ['SPAL 2013', 'VS', 'Inter'], ['Leeds', 'VS', 'Barnsley'], ['Rubin Kazan', 'VS', 'FC Rostov'], ['FC Ufa', 'VS', 'Dinamo Moscow'], ['Lokomotiv Moscow', 'VS', 'CSKA Moscow'], ['FC Tambov', 'VS', 'PFC Sochi'], ['Dynamo Kyiv', 'VS', 'Zorya']]


In [14]:
game=BeautifulSoup('<tr class="item ls-4 stream-is-available" id="i1458601" data-group-id="18478"><td class="ls-s selection"><input type="checkbox" id="is1458601"></td><td class="toolbar left"></td><td class="time">17:00</td><td class="status"><span class="status-4 rc"></span></td><td class="topmatch-column"></td><td class="team home"><a class="team-link" href="/Teams/350/Show/Ukraine-Dynamo-Kyiv"><span class="team-name">Dynamo Kyiv</span></a></td><td class="result"><a class="livestream-link" href="/Matches/1458601/LiveStream/Ukraine-Premier-League-2019-2020-Dynamo-Kyiv-Zorya">Watch Live</a><a class="result-4 rc" href="/Matches/1458601/Show/Ukraine-Premier-League-2019-2020-Dynamo-Kyiv-Zorya">vs</a></td><td class="team away"><a class="team-link" href="/Teams/3753/Show/Ukraine-Zorya"><span class="team-name">Zorya</span></a></td><td class="stage ls-t"><a href="/Regions/230/Tournaments/114/Seasons/7813/Stages/18478/Show/Ukraine-Premier-League-2019-2020" class="iconize iconize-icon-left"><span class="ui-icon country flg-ua"></span>UPL</a></td><td class="toolbar right"><div style="width:12.75em;"><a href="/Matches/1458601/LiveStream/Ukraine-Premier-League-2019-2020-Dynamo-Kyiv-Zorya" class="iconize "><span class="incidents-icon ui-icon stream">Stream</span></a></div></td></tr>', 'html.parser')
gamesToReport.append([game.find('td', class_ = 'team home').a.getText(),"VS",game.find('td', class_ = 'team away').a.getText()])
gameTFile.append(getTeamInfo(game.find('td', class_ = 'team home')) + getTeamInfo(game.find('td', class_ = 'team away')))

['6.82', 2, 0, 2, 1, 2, 3, 2, 3, 1, 1, 2, 2, '1.2', '60.9', '83.8', '18.3', '6.2', '6.6', '7.1', '7', '7.6', '7.2', '7.3', '7', '6.5', '7.2', '6.9']
['6.54', 1, 0, 1, 3, 2, 1, 0, 0, 2, 2, 2, 0, '0.5', '47.9', '72.8', '11']
